In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests
import re
import io
#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

In [2]:
def extract_number(value: str) -> float:
    matches = re.findall(r"\$*(\d{1,}[\.\d{1,}]*)", value)

    if matches:
        return sum(float(match) for match in matches)/len(matches)
            
    return None
 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}   

# Q1

In [3]:
response = requests.get("https://stockanalysis.com/ipos/filings/", headers=headers)

ipo_dfs = pd.read_html(response.text)[0]

/tmp/ipykernel_71646/2515200017.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)[0]


In [4]:
ipo_dfs.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
3,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
4,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-


In [5]:
ipo_dfs["Filing Date"] = pd.to_datetime(ipo_dfs["Filing Date"])
ipo_dfs['Avg_price'] = ipo_dfs["Price Range"].apply(extract_number)
ipo_dfs["Shares Offered"] = ipo_dfs["Shares Offered"].apply(extract_number)

In [6]:
ipo_dfs["Shares_offered_value"] = ipo_dfs['Avg_price'] * ipo_dfs["Shares Offered"]

In [7]:
ipo_dfs.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
1,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
2,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.0,NaN


In [8]:
round(ipo_dfs[(ipo_dfs["Filing Date"].dt.year == 2023) & (ipo_dfs["Filing Date"].dt.dayofweek == 4)]["Shares_offered_value"].sum() / 1_000_000)

286

# Q2

In [9]:
response_2023 = requests.get("https://stockanalysis.com/ipos/2023/", headers=headers)
response_2024 = requests.get("https://stockanalysis.com/ipos/2024/", headers=headers)

ipo_dfs = pd.concat(
    [pd.read_html(io.StringIO(str(response_2024.text)))[0], pd.read_html(io.StringIO(str(response_2023.text)))[0]],
    ignore_index=True
)

In [10]:
ipo_dfs["IPO Date"] = pd.to_datetime(ipo_dfs['IPO Date'], format='mixed')
ipo_dfs["IPO Price"] = ipo_dfs["IPO Price"].apply(extract_number)
ipo_dfs["Current"] = ipo_dfs["Current"].apply(extract_number)
ipo_dfs["Return"] = ipo_dfs["Return"].apply(lambda x: float(x.replace("%", ""))/100 if x != "-" else None)

In [11]:
ipo_before = ipo_dfs[ipo_dfs["IPO Date"] < "2024-03-01"]

In [12]:
ipo_before

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",4.00,10.43,1.5025
34,2024-02-22,VHAI,Vocodia Holdings Corp,4.25,0.16,-0.9614
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,10.00,10.16,0.0150
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,6.00,1.76,-0.6992
37,2024-02-14,UMAC,"Unusual Machines, Inc.",4.00,1.20,-0.7000
...,...,...,...,...,...,...
213,2023-01-25,QSG,QuantaSing Group Ltd,12.50,3.06,-0.7592
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",5.00,0.43,-0.9120
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",15.00,35.28,1.3560
216,2023-01-13,ISRL,Israel Acquisitions Corp,10.00,10.93,0.0910


In [13]:
symbols = ipo_before["Symbol"].to_list()

In [14]:
data = {}

for symbol in symbols:
    try:
        history = yf.download(tickers = symbol, period = "max", interval = "1d")
        data[symbol] = history
    except Exception as e:
        print(f'{symbol} -> {e}')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [15]:
data["PTHR"] = yf.download(tickers = "PTHRU", period = "max", interval = "1d") 

[*********************100%%**********************]  1 of 1 completed


In [16]:
data["RYZB"] = yf.download(tickers = "RYZB", period = "max", interval = "1d") 

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['RYZB']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [17]:
def compute_growths(df: pd.DataFrame) -> pd.DataFrame:
    for i in range(1, 31):
        df[f'growth_future_{i}d'] = df['Adj Close'].shift(-i) / df['Adj Close']
    return df

In [18]:
blacklist = ["RYZB", "DYCQ", "LEGT"]

In [19]:
for key, value in data.items():
    if key in blacklist:
        continue
    compute_growths(value)

In [20]:
first_day_price = []
for key, value in data.items():
    if key in blacklist:
        continue

    first_day_price.append({**value.iloc[0].to_dict(), **{"Symbol": key}})

first_day_price = pd.DataFrame(first_day_price)

In [21]:
describe = first_day_price.describe()

In [22]:
describe[describe.index == "75%"].filter(regex='growth_future').idxmax(axis=1)

75%    growth_future_28d
dtype: object

# Q3

In [23]:
def clean_up(history: pd.DataFrame, start: str = "2014-01-01", end: str = "2023-12-31", growth: int = 7) -> pd.DataFrame:
    history[f'growth_{growth}d'] = history['Adj Close'] / history['Adj Close'].shift(growth) 
    return history[(history.index <= end) & (history.index >= start)]


In [24]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [25]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [26]:
data_largest = {}
for symbol in LARGEST_STOCKS:
    history = yf.download(tickers = symbol, period = "max", interval = "1d")
    data_largest[symbol] = history

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [27]:
data_large = {}
for symbol in LARGE_STOCKS:
    history = yf.download(tickers = symbol, period = "max", interval = "1d")
    data_large[symbol] = history

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [59]:
large_7day = []
for key, value in data_large.items():
    value = clean_up(value)
    large_7day.append(value["growth_7d"])

In [60]:
large_df = pd.concat(large_7day, axis=1).mean(axis=1)

In [61]:
largest_7day = []
for key, value in data_largest.items():
    value = clean_up(value)
    largest_7day.append(value["growth_7d"])

In [62]:
largest_df = pd.concat(largest_7day, axis=1).mean(axis=1)

In [63]:
largest_df.iloc[0], large_df.iloc[0]

(1.0117971428666503, 1.0092881302610848)

In [64]:
comparision = (large_df > largest_df)

In [71]:
round(100*comparision.sum() / len(comparision))

47

# Q4

In [74]:
import pyarrow.parquet as pq
table = pq.read_table('stocks_df_combined_trunc_2014_2023.parquet.brotli')
df = table.to_pandas()

In [77]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=202)

In [83]:
df_cci_200 = df[(df["cci"] > 200) & (df["Weekday"] == 4)]
df_cci_200

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7113,40.450001,40.970001,40.250000,40.939999,34.912762,34567600.0,MSFT,2014,2014-05-01,4,...,0.997447,1.005607,1.021664,0.995813,NaN,NaN,NaN,NaN,NaN,NaN
7181,45.110001,45.930000,45.110001,45.910000,39.395618,36939400.0,MSFT,2014,2014-09-01,4,...,0.983610,0.941627,0.925124,0.914716,NaN,NaN,NaN,NaN,NaN,NaN
7340,45.660000,48.139999,45.650002,47.869999,41.630741,130933700.0,MSFT,2015,2015-04-01,4,...,1.082228,1.143658,1.055457,0.614516,0.978035,0.982994,1.037625,0.939362,0.933108,NaN
7452,44.270000,45.570000,43.919998,45.570000,40.151123,41839000.0,MSFT,2015,2015-10-01,4,...,1.007958,1.032390,0.775540,0.441114,0.998922,1.002560,1.009139,1.034930,0.909566,0.632660
7467,52.299999,54.070000,52.250000,52.869999,46.583046,135227100.0,MSFT,2015,2015-10-01,4,...,0.976399,0.996884,0.751370,0.435520,1.009025,1.026100,1.051840,1.200679,0.957738,0.771437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072,2061.000000,2095.800049,2058.449951,2062.750000,2058.108887,2652761.0,LT.NS,2022,2022-11-01,4,...,0.891008,0.884318,0.782174,1.124210,0.995024,1.020511,0.989464,0.795450,0.824372,0.288467
5219,2420.000000,2483.500000,2415.050049,2475.550049,2469.979980,2690699.0,LT.NS,2023,2023-06-01,4,...,0.971214,0.973233,0.901866,0.870323,1.001048,0.993127,0.992891,1.119678,1.072726,1.540443
5234,2522.000000,2595.000000,2521.100098,2586.250000,2580.430908,4610417.0,LT.NS,2023,2023-07-01,4,...,1.011984,1.053541,1.003714,0.869197,1.003918,1.001748,0.985979,0.996052,1.075177,1.291138
5268,2888.000000,2928.699951,2872.449951,2901.600098,2901.600098,3638510.0,LT.NS,2023,2023-09-01,4,...,1.060358,1.076092,1.142983,0.751347,0.987251,1.004875,1.004067,0.876331,1.002105,1.340190


In [107]:
total_sum = 0.0
investment = 1000

for idx, row in df_cci_200.iterrows():
    try:
        df_ticker = df[df["Ticker"] == row.Ticker]     
        buy_price = row["Adj Close_x"]
        sell_price = df_ticker[df_ticker.index == idx+5]["Adj Close_x"]

        total_sum += (investment/buy_price) * sell_price.values[0] - investment
    except Exception as e:
        print(f"Error: {e} skipping {idx}")

Error: index 0 is out of bounds for axis 0 with size 0 skipping 5643


In [110]:
int(total_sum / 1000)

1